In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from keras.models import Model
from keras.layers import Flatten, Dense

c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SEED = 1987
data_dir = ''
labels = pd.read_csv(join(data_dir, 'labels.csv'))
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


In [3]:
# This part is optional for if you want to train only on most popular classes. 
# NUM_CLASSES = 16
# selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
# labels = labels[labels['breed'].isin(selected_breed_list)]

In [4]:
labels['target'] = 1
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
y_train = labels_pivot.values
ytr = y_train[train_idx]
yv = y_train[valid_idx]

In [5]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [6]:
INPUT_SIZE = 320
x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

10222it [00:56, 179.46it/s]


Train Images shape: (10222, 320, 320, 3) size: 3,140,198,400


In [7]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))

((8140, 320, 320, 3), (2082, 320, 320, 3), (8140, 121), (2082, 121))


In [8]:
del x_train

In [9]:
model_resnet50 = ResNet50(weights='imagenet',include_top=False, input_shape = (INPUT_SIZE,INPUT_SIZE,3))
x = model_resnet50.get_layer(model_resnet50.layers[-2].name).output
x = Flatten(name="Last_flatterning")(x)
x = Dense(128, activation='relu', name='Last_Relu')(x)
predictions = Dense(ytr.shape[1]-1, activation='softmax',name='Last_softmax')(x) 
model = Model(inputs=model_resnet50.input, outputs=predictions)

In [10]:
for i, layer in enumerate(model.layers[:-3]):
   layer.trainable = False
   #print(i, layer.name,layer.trainable)   

In [11]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics = ["accuracy"])

In [12]:
model.fit(x = Xtr, y = ytr[:,1:], epochs = 1, batch_size = 32,validation_data = [Xv,yv[:,1:]])

Train on 8140 samples, validate on 2082 samples
Epoch 1/1


6656/8140 [=======================>......] - ETA: 15:18 - loss: 0.0640 - acc: 0.99 - ETA: 8:23 - loss: 0.1499 - acc: 0.9878 - ETA: 6:04 - loss: 0.1650 - acc: 0.986 - ETA: 4:55 - loss: 0.1881 - acc: 0.985 - ETA: 4:13 - loss: 0.1974 - acc: 0.985 - ETA: 3:45 - loss: 0.2076 - acc: 0.985 - ETA: 3:25 - loss: 0.2149 - acc: 0.985 - ETA: 3:10 - loss: 0.2204 - acc: 0.984 - ETA: 2:58 - loss: 0.2256 - acc: 0.984 - ETA: 2:48 - loss: 0.2298 - acc: 0.984 - ETA: 2:40 - loss: 0.2332 - acc: 0.984 - ETA: 2:33 - loss: 0.2360 - acc: 0.984 - ETA: 2:27 - loss: 0.2384 - acc: 0.984 - ETA: 2:23 - loss: 0.2405 - acc: 0.984 - ETA: 2:18 - loss: 0.2422 - acc: 0.984 - ETA: 2:15 - loss: 0.2438 - acc: 0.984 - ETA: 2:11 - loss: 0.2452 - acc: 0.984 - ETA: 2:08 - loss: 0.2464 - acc: 0.984 - ETA: 2:05 - loss: 0.2475 - acc: 0.984 - ETA: 2:03 - loss: 0.2485 - acc: 0.983 - ETA: 2:01 - loss: 0.2494 - acc: 0.983 - ETA: 1:58 - loss: 0.2494 - acc: 0.983 - ETA: 1:56 - loss: 0.2495 - acc: 0.983 - ETA: 1:54 - loss: 0.2502 - acc: 0.

8140/8140 [==============================] - ETA: 16s - loss: 0.2625 - acc: 0.98 - ETA: 16s - loss: 0.2625 - acc: 0.98 - ETA: 16s - loss: 0.2625 - acc: 0.98 - ETA: 15s - loss: 0.2625 - acc: 0.98 - ETA: 15s - loss: 0.2625 - acc: 0.98 - ETA: 15s - loss: 0.2625 - acc: 0.98 - ETA: 14s - loss: 0.2625 - acc: 0.98 - ETA: 14s - loss: 0.2625 - acc: 0.98 - ETA: 13s - loss: 0.2626 - acc: 0.98 - ETA: 13s - loss: 0.2626 - acc: 0.98 - ETA: 13s - loss: 0.2625 - acc: 0.98 - ETA: 12s - loss: 0.2626 - acc: 0.98 - ETA: 12s - loss: 0.2626 - acc: 0.98 - ETA: 12s - loss: 0.2626 - acc: 0.98 - ETA: 11s - loss: 0.2626 - acc: 0.98 - ETA: 11s - loss: 0.2626 - acc: 0.98 - ETA: 10s - loss: 0.2627 - acc: 0.98 - ETA: 10s - loss: 0.2627 - acc: 0.98 - ETA: 10s - loss: 0.2627 - acc: 0.98 - ETA: 9s - loss: 0.2627 - acc: 0.9835 - ETA: 9s - loss: 0.2627 - acc: 0.983 - ETA: 9s - loss: 0.2628 - acc: 0.983 - ETA: 8s - loss: 0.2628 - acc: 0.983 - ETA: 8s - loss: 0.2628 - acc: 0.983 - ETA: 7s - loss: 0.2628 - acc: 0.983 - ETA:

In [13]:
for i, layer in enumerate(model.layers):
   layer.trainable = True

In [16]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics = ["accuracy"])
model.fit(x = Xtr, y = ytr[:,1:], epochs = 1, batch_size = 8,validation_data = [Xv,yv[:,1:]])

Train on 8140 samples, validate on 2082 samples
Epoch 1/1


1624/8140 [====>.........................] - ETA: 2:49:51 - loss: 0.1847 - acc: 0.98 - ETA: 1:27:55 - loss: 0.1849 - acc: 0.98 - ETA: 1:00:37 - loss: 0.1941 - acc: 0.98 - ETA: 46:56 - loss: 0.2014 - acc: 0.9857 - ETA: 38:46 - loss: 0.1702 - acc: 0.98 - ETA: 33:17 - loss: 0.1679 - acc: 0.98 - ETA: 29:22 - loss: 0.1640 - acc: 0.98 - ETA: 26:25 - loss: 0.1584 - acc: 0.98 - ETA: 24:09 - loss: 0.1583 - acc: 0.98 - ETA: 22:18 - loss: 0.1518 - acc: 0.98 - ETA: 20:48 - loss: 0.1449 - acc: 0.98 - ETA: 19:33 - loss: 0.1414 - acc: 0.98 - ETA: 18:28 - loss: 0.1384 - acc: 0.98 - ETA: 17:33 - loss: 0.1339 - acc: 0.98 - ETA: 16:46 - loss: 0.1341 - acc: 0.98 - ETA: 16:04 - loss: 0.1330 - acc: 0.98 - ETA: 15:27 - loss: 0.1296 - acc: 0.98 - ETA: 14:55 - loss: 0.1266 - acc: 0.98 - ETA: 14:26 - loss: 0.1240 - acc: 0.98 - ETA: 14:01 - loss: 0.1229 - acc: 0.98 - ETA: 13:37 - loss: 0.1207 - acc: 0.98 - ETA: 13:15 - loss: 0.1187 - acc: 0.98 - ETA: 12:55 - loss: 0.1168 - acc: 0.98 - ETA: 12:37 - loss: 0.1151 -

3256/8140 [===========>..................] - ETA: 5:31 - loss: 0.0937 - acc: 0.989 - ETA: 5:30 - loss: 0.0936 - acc: 0.989 - ETA: 5:30 - loss: 0.0936 - acc: 0.989 - ETA: 5:29 - loss: 0.0936 - acc: 0.989 - ETA: 5:28 - loss: 0.0936 - acc: 0.989 - ETA: 5:28 - loss: 0.0935 - acc: 0.989 - ETA: 5:27 - loss: 0.0935 - acc: 0.989 - ETA: 5:27 - loss: 0.0936 - acc: 0.989 - ETA: 5:26 - loss: 0.0937 - acc: 0.989 - ETA: 5:25 - loss: 0.0938 - acc: 0.989 - ETA: 5:25 - loss: 0.0938 - acc: 0.989 - ETA: 5:24 - loss: 0.0938 - acc: 0.989 - ETA: 5:24 - loss: 0.0937 - acc: 0.989 - ETA: 5:23 - loss: 0.0937 - acc: 0.989 - ETA: 5:23 - loss: 0.0937 - acc: 0.989 - ETA: 5:22 - loss: 0.0936 - acc: 0.989 - ETA: 5:21 - loss: 0.0936 - acc: 0.989 - ETA: 5:21 - loss: 0.0935 - acc: 0.989 - ETA: 5:20 - loss: 0.0934 - acc: 0.989 - ETA: 5:20 - loss: 0.0933 - acc: 0.989 - ETA: 5:19 - loss: 0.0934 - acc: 0.989 - ETA: 5:18 - loss: 0.0933 - acc: 0.989 - ETA: 5:18 - loss: 0.0932 - acc: 0.989 - ETA: 5:17 - loss: 0.0933 - acc: 0.9

4888/8140 [=================>............] - ETA: 3:53 - loss: 0.0888 - acc: 0.990 - ETA: 3:53 - loss: 0.0888 - acc: 0.990 - ETA: 3:52 - loss: 0.0887 - acc: 0.990 - ETA: 3:52 - loss: 0.0887 - acc: 0.990 - ETA: 3:52 - loss: 0.0887 - acc: 0.990 - ETA: 3:51 - loss: 0.0887 - acc: 0.990 - ETA: 3:51 - loss: 0.0887 - acc: 0.990 - ETA: 3:50 - loss: 0.0887 - acc: 0.990 - ETA: 3:50 - loss: 0.0887 - acc: 0.990 - ETA: 3:50 - loss: 0.0886 - acc: 0.990 - ETA: 3:49 - loss: 0.0885 - acc: 0.990 - ETA: 3:49 - loss: 0.0885 - acc: 0.990 - ETA: 3:48 - loss: 0.0886 - acc: 0.990 - ETA: 3:48 - loss: 0.0886 - acc: 0.990 - ETA: 3:48 - loss: 0.0886 - acc: 0.990 - ETA: 3:47 - loss: 0.0887 - acc: 0.990 - ETA: 3:47 - loss: 0.0887 - acc: 0.990 - ETA: 3:46 - loss: 0.0886 - acc: 0.990 - ETA: 3:46 - loss: 0.0886 - acc: 0.990 - ETA: 3:46 - loss: 0.0886 - acc: 0.990 - ETA: 3:45 - loss: 0.0885 - acc: 0.990 - ETA: 3:45 - loss: 0.0885 - acc: 0.990 - ETA: 3:44 - loss: 0.0885 - acc: 0.990 - ETA: 3:44 - loss: 0.0884 - acc: 0.9

6520/8140 [=======================>......] - ETA: 2:32 - loss: 0.0857 - acc: 0.990 - ETA: 2:32 - loss: 0.0857 - acc: 0.990 - ETA: 2:32 - loss: 0.0856 - acc: 0.990 - ETA: 2:31 - loss: 0.0856 - acc: 0.990 - ETA: 2:31 - loss: 0.0855 - acc: 0.990 - ETA: 2:30 - loss: 0.0855 - acc: 0.990 - ETA: 2:30 - loss: 0.0854 - acc: 0.990 - ETA: 2:30 - loss: 0.0853 - acc: 0.990 - ETA: 2:29 - loss: 0.0853 - acc: 0.990 - ETA: 2:29 - loss: 0.0852 - acc: 0.990 - ETA: 2:28 - loss: 0.0852 - acc: 0.990 - ETA: 2:28 - loss: 0.0851 - acc: 0.990 - ETA: 2:28 - loss: 0.0850 - acc: 0.990 - ETA: 2:27 - loss: 0.0850 - acc: 0.990 - ETA: 2:27 - loss: 0.0849 - acc: 0.990 - ETA: 2:26 - loss: 0.0849 - acc: 0.990 - ETA: 2:26 - loss: 0.0848 - acc: 0.990 - ETA: 2:26 - loss: 0.0848 - acc: 0.990 - ETA: 2:25 - loss: 0.0847 - acc: 0.990 - ETA: 2:25 - loss: 0.0846 - acc: 0.990 - ETA: 2:25 - loss: 0.0846 - acc: 0.990 - ETA: 2:24 - loss: 0.0845 - acc: 0.990 - ETA: 2:24 - loss: 0.0845 - acc: 0.990 - ETA: 2:23 - loss: 0.0844 - acc: 0.9

8140/8140 [==============================] - ETA: 1:15 - loss: 0.0765 - acc: 0.990 - ETA: 1:14 - loss: 0.0765 - acc: 0.990 - ETA: 1:14 - loss: 0.0764 - acc: 0.990 - ETA: 1:14 - loss: 0.0764 - acc: 0.990 - ETA: 1:13 - loss: 0.0764 - acc: 0.990 - ETA: 1:13 - loss: 0.0763 - acc: 0.990 - ETA: 1:12 - loss: 0.0763 - acc: 0.990 - ETA: 1:12 - loss: 0.0763 - acc: 0.990 - ETA: 1:12 - loss: 0.0762 - acc: 0.990 - ETA: 1:11 - loss: 0.0762 - acc: 0.990 - ETA: 1:11 - loss: 0.0762 - acc: 0.990 - ETA: 1:11 - loss: 0.0761 - acc: 0.990 - ETA: 1:10 - loss: 0.0761 - acc: 0.990 - ETA: 1:10 - loss: 0.0761 - acc: 0.990 - ETA: 1:09 - loss: 0.0760 - acc: 0.990 - ETA: 1:09 - loss: 0.0760 - acc: 0.990 - ETA: 1:09 - loss: 0.0760 - acc: 0.990 - ETA: 1:08 - loss: 0.0759 - acc: 0.990 - ETA: 1:08 - loss: 0.0759 - acc: 0.990 - ETA: 1:08 - loss: 0.0759 - acc: 0.990 - ETA: 1:07 - loss: 0.0758 - acc: 0.990 - ETA: 1:07 - loss: 0.0758 - acc: 0.990 - ETA: 1:06 - loss: 0.0758 - acc: 0.990 - ETA: 1:06 - loss: 0.0757 - acc: 0.9

In [17]:
model.save_weights('trained_weights_resenet50.h5')